<center>
<h2> </h2>
<h2>ALTeGraD 2024<br>
<h2>Lab Session 3: Large Language Models</h2>
<h5>October 22, 2024</h5>
<h4><b>Student Name:</b> NAIM Mouna</h4> 
<br>
</center>
[Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B)
<hr style="border:10px solid gray"> </hr>
<p style="text-align: justify;">
This handout includes theoretical introductions, <font color='blue'>coding tasks</font> and <font color='red'>questions</font>. Before the deadline, you should submit a <B>.ipynb</B> file named <b>Lastname_Firstname.ipynb</b> containing your notebook (with the gaps filled and your answers to the questions). Your answers should be well constructed and well justified. They should not repeat the question or generalities in the handout. When relevant, you are welcome to include figures, equations and tables derived from your own computations, theoretical proofs or qualitative explanations. One submission is required for each student. The deadline for this lab is <b>October 29, 2024 11:59 PM</b>. No extension will be granted. Late policy is as follows: ]0, 24] hours late → -5 pts; ]24, 48] hours late → -10 pts; > 48 hours late → not graded (zero).

<hr style="border:5px solid gray"> </hr>

<hr style="border:5px solid gray"> </hr>

* Please submit your file to Moodle or [here](https://docs.google.com/forms/d/e/1FAIpQLSfDFjQcvjKxLn42Kxpw5ek2Ce_lHzMCVSST8R2AJcQct6Np2A/viewform)

<br><br>
In this lab, we will:

* fintune [Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B) on a question/answer dataset.

* To reduce the required GPU VRAM for the finetuning, we will use [LoRA](https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2) and [quantization](https://huggingface.co/blog/4bit-transformers-bitsandbytes) techniques.

* Compare the results before and after instructin tuning.

* Fintune the model again on perference dataset using [DPO](https://huggingface.co/docs/trl/main/dpo_trainer#dpo-trainer)(direct perference optimization)
 <br>


# <b>Part 1 Finetuning Qwen2.5-0.5B using HuggingFace's Transfromers</b>
In this section, we will fintune [Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B) on a question/answer dataset.

To reduce the required GPU VRAM for the finetuning, we will use [LoRA](https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2) and [quantization](https://huggingface.co/blog/4bit-transformers-bitsandbytes) techniques.

## <b>Preparing the environment and installing libraries:<b>

In [3]:
!nvidia-smi

Sun Oct 27 14:10:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.33                 Driver Version: 546.33       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   45C    P0              17W /  75W |      0MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -qqq bitsandbytes torch transformers peft accelerate datasets loralib einops trl

In [ ]:
!pip install --upgrade transformers==4.45.2

In [4]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer

from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

## <b>Loading the model and the tokenizer:<b>

In this section, we will load the QWEN model while using the BitsAndBytes library for quantization.

In [5]:
#MODEL_NAME = "Qwen/Qwen2.5-0.5B"
MODEL_NAME = "unsloth/Llama-3.2-1B" # Try Llama if you want

# fill the gap (filled)
# We will use the function "BitsAndBytesConfig" to configure quantization parameters 
# To reduce VRAM usage, we will use 4bit precision
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    #bnb_4bit_compute_dtype=torch.bfloat16,
    #bnb_4bit_quant_type="nf4", 
    bnb_4bit_use_double_quant=True
) 

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    #force_download=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

In [6]:
def print_trainable_parameters(model):

    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        # fill the gap: get the number of trainable parameters: trainable_params
        # We will check if the parameter is trainable
        if param.requires_grad:  
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

## <b>Configuring LoRA:<b>

In [7]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    #target_modules=["query_key_value"],
    bias="none",
    task_type= "CAUSAL_LM"# fill the gaph
)

model = get_peft_model(model, config) # fill the gap, using lora weights
print_trainable_parameters(model)

trainable params: 1703936 || all params: 750979072 || trainable%: 0.22689527092440734


## <b>Test the model before finetuning:<b>

In [8]:
prompt = "<human>: What equipment do I need for rock climbing?\n<assistant>: "
 # # fill the gap, prompt of the format: "<human>: What equipment do I need for rock climbing?  \n <assistant>: ", with an empty response from the assistant
print(prompt)


generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
#generation_config.do_sample = True

<human>: What equipment do I need for rock climbing?
<assistant>: 


Question: what does the temperature do in the above cell?

### Answer 
Temperature adjusts the randomness of a model's output. When the temperature is set below 1, the model becomes more deterministic, favoring more predictable and likely next tokens. Conversely, if the temperature is set above 1, the output becomes more random as the model is encouraged to explore less likely words, which can lead to a greater variety but may also introduce errors.

In [11]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

c:\Users\red-y\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\red-y\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<human>: What equipment do I need for rock climbing?
<assistant>: 1. What is the best equipment for rock climbing?
<assistant>: 2. What is the best equipment for rock climbing?
<assistant>: 3. What is the best equipment for rock climbing?
<assistant>: 4. What is the best equipment for rock climbing?
<assistant>: 5. What is the best equipment for rock climbing?
<assistant>: 6. What is the best equipment for rock climbing?
<assistant>: 7. What is the best equipment for rock climbing?
<assistant>: 8. What is the best equipment for rock climbing?
<assistant>: 9. What is the best equipment for rock climbing?
<assistant>: 10. What is the best equipment for rock climbing?
<assistant>: 11. What is the best equipment for rock climbing?
<assistant>: 12. What is the best equipment for rock climbing?
<assistant>: 13. What is the best equipment for rock climbing?
<assistant>: 14. What is the
CPU times: total: 12.5 s
Wall time: 13.8 s


## <b>Loading the question/answer dataset from HuggingFace:<b>

In [10]:
data = load_dataset("HuggingFaceH4/helpful-instructions")
pd.DataFrame(data["train"])

,instruction,demonstration,meta
0,"Hi, I want to learn to play horseshoes. Can yo...","I can, but maybe I should begin by telling you...",{'source': 'helpful-anthropic-raw'}
1,How do I teach kids to meditate?,Great question! That’s a really useful skill t...,{'source': 'helpful-anthropic-raw'}
2,Can you tell me the steps for getting a harbor...,Sure. I believe you’ll need a copy of the mari...,{'source': 'helpful-anthropic-raw'}
3,How can I store food if I don't have a pantry?,"You could store the food in a refrigerator, th...",{'source': 'helpful-anthropic-raw'}
4,what are some good novels for a 9 year old?,"That depends on the 9 year old, but if they li...",{'source': 'helpful-anthropic-raw'}
...,...,...,...
147701,"Given the following sentence, classify it into...",Fact,{'source': 'helpful-self-instruct-raw'}
147702,A person wants to write a book. he/she writes ...,Chapter 1 - The History of China\nChapter 2 - ...,{'source': 'helpful-self-instruct-raw'}
147703,Tell me how you would make a popular app game.,I would make a game that is similar to 2048. T...,{'source': 'helpful-self-instruct-raw'}
147704,Describe your dream house to me.\n\nOutput:,My dream house is a two-story building with a ...,{'source': 'helpful-self-instruct-raw'}


## <b>Preparing the finetuning data:<b>

In [12]:
def generate_prompt(data_point):
    question = data_point['instruction']
    response = data_point['demonstration']
    return f"<human>: {question}\n<assistant>: {response}"
    # fill the gap, transform the data into prompts of the format: "<human>: question?  \n <assistant>: response"

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

data = data["train"].shuffle(seed=42).map(generate_and_tokenize_prompt)

## <b>Finetuning:<b>

In [13]:
OUTPUT_DIR = "experiments"

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=300,   # try more steps if you can
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

max_steps is given, it will override any value given in num_train_epochs


  0%|          | 0/300 [00:00<?, ?it/s]

c:\Users\red-y\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\autograd\graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\MHA.cpp:676.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


{'loss': 3.5725, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.0}
{'loss': 2.4003, 'grad_norm': 1.5642026662826538, 'learning_rate': 1.3333333333333333e-05, 'epoch': 0.0}
{'loss': 2.8986, 'grad_norm': 1.774344801902771, 'learning_rate': 2.6666666666666667e-05, 'epoch': 0.0}
{'loss': 2.5586, 'grad_norm': 1.654936671257019, 'learning_rate': 4e-05, 'epoch': 0.0}
{'loss': 3.0194, 'grad_norm': 1.9287158250808716, 'learning_rate': 5.333333333333333e-05, 'epoch': 0.0}
{'loss': 2.6012, 'grad_norm': 1.0093914270401, 'learning_rate': 6.666666666666667e-05, 'epoch': 0.0}
{'loss': 3.0654, 'grad_norm': 2.0078952312469482, 'learning_rate': 8e-05, 'epoch': 0.0}
{'loss': 2.5509, 'grad_norm': 2.088275909423828, 'learning_rate': 9.333333333333334e-05, 'epoch': 0.0}
{'loss': 2.9975, 'grad_norm': 2.637000322341919, 'learning_rate': 0.00010666666666666667, 'epoch': 0.0}
{'loss': 2.8589, 'grad_norm': 2.1615777015686035, 'learning_rate': 0.00012, 'epoch': 0.0}
{'loss': 2.4632, 'grad_norm': 1.62770652770

TrainOutput(global_step=300, training_loss=1.997491771777471, metrics={'train_runtime': 514.3682, 'train_samples_per_second': 2.333, 'train_steps_per_second': 0.583, 'total_flos': 619367078596608.0, 'train_loss': 1.997491771777471, 'epoch': 0.008124246814618229})

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir experiments/runs --port 6008

## <b>Test the model after the finetuning:<b>

In [14]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

c:\Users\red-y\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\red-y\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<human>: What equipment do I need for rock climbing?
<assistant>: 1. A helmet. 2. A harness. 3. A rope. 4. A pair of climbing shoes. 5. A climbing wall. 6. A climbing guide. 7. A climbing instructor. 8. A climbing instructor’s guide. 9. A climbing instructor’s book. 10. A climbing instructor’s course. 11. A climbing instructor’s certificate. 12. A climbing instructor’s license. 13. A climbing instructor’s license. 14. A climbing instructor’s license. 15. A climbing instructor’s license. 16. A climbing instructor’s license. 17. A climbing instructor’s license. 18. A climbing instructor’s license. 19. A climbing instructor’s license. 20. A climbing instructor’s license. 21. A climbing instructor’s license. 22. A climbing instructor’s license. 23. A climbing instructor’s license. 24. A climbing instructor’s license.
CPU times: total: 12.4 s
Wall time: 14.3 s


In [15]:
def generate_response(question: str) -> str:
    prompt = f"<human>: {question}\n<assistant>: " # fill the gap, transform the data into prompts of the format: "<human>: question?  \n <assistant>: " with an empty response
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [16]:
prompt = "What program can I use to edit video clips I took with my phone?"
print('-', prompt,'\n')
print(generate_response(prompt))

prompt = "Do you know the reasons as to why people love coffee so much?"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))



- What program can I use to edit video clips I took with my phone? 

I’m not sure what you mean by “edit video clips”.  Can you explain what you mean?  Are you trying to edit the video clips so that they are more visually appealing?  Are you trying to edit the video clips so that they are more informative?  Are you trying to edit the video clips so that they are more entertaining?  Are you trying to edit the video clips so that they are more engaging?  Are you trying to edit the video clips so that they are more informative?  Are you trying to edit the video clips so that they are more entertaining?  Are you trying to edit the video clips so that they are more engaging?  Are you trying to edit the video clips so that they are more informative?  Are you trying to edit the video clips so that they are more entertaining?  Are you trying to edit the video clips so that they are more engaging?  Are you trying to edit the video clips so that they are more informative?  Are



- Do you know t

# Part 2: DPO
In this part we will use the instrcution tuned LLM to do direct preference optimization. see the paper: https://arxiv.org/abs/2305.18290

DPO involves tuning the model on preference data, normally consists of a prompt, a prefered answer and a rejected answer.

The core advantage of DPO is its ability to simultaneously bypass the explicit reward modeling step while avoiding the complexities of reinforcement learning optimization.

## Test the model before DPO:


In [17]:
prompt_2 =f"<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  \n<assistant>: " # fill the gap, prompt of the format: "<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  \n <assistant>: ", with an empty response from the assistant
print(prompt_2)

<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  
<assistant>: 


In [18]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt_2, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  
<assistant>:  Yes, I can taste it.  It needs more spices.  I think it would be better if you added some chili powder.  It's not too spicy, but it could be better.  I think you should add some more chili powder.  It's not too spicy, but it could be better.  I think you should add some more chili powder.  It's not too spicy, but it could be better.  I think you should add some more chili powder.  It's not too spicy, but it could be better.  I think you should add some more chili powder.  It's not too spicy, but it could be better.  I think you should add some more chili powder.  It's not too spicy, but it could be better.  I think you should add some more chili powder.  It's not too spicy, but it could be better.  I think you should add some more chili powder.  It's not too
CPU times: total: 17.1 s
Wall time: 18.8 s


## Loading the preference data from Huggingface:

In [52]:
data_dpo = load_dataset("CultriX/llama70B-dpo-dataset")
pd.DataFrame(data_dpo["train"])

,system,question,chosen,rejected
0,You are a helpful assistant,Can you taste this dish and tell me if it need...,I'm not capable of physically tasting the dish...,I can taste the dish remotely and I think it n...
1,You are a helpful assistant,Can you smell this perfume and tell me if it's...,"As a digital AI assistant, I don't have a phys...","I can smell the perfume, and it smells great. ..."
2,You are a helpful assistant,Please turn off the lights in my room.,"I'm a large language model, I don't have the c...",I can turn off the lights in your room remotel...
3,You are a helpful assistant,What is the primary reason why many approximat...,The primary reason why many approximate attent...,Approximate attention methods haven't gained w...
4,You are a helpful assistant,What are the properties of melanin that contri...,Melanin's photoprotective role can be attribut...,Melanin's photoprotective role is due to its a...
...,...,...,...,...
2174,You are a helpful assistant,What is the sound barrier?,"The sound barrier, also known as the sonic bar...",The sound barrier is a physical wall that prev...
2175,You are a helpful assistant,Is spinach a great source of dietary iron?,"Spinach is a good source of dietary iron, but ...","Spinach is a poor source of dietary iron, and ..."
2176,You are a helpful assistant,What challenges arise in training large langua...,Training large language models (LLMs) poses se...,The main challenge in training LLMs is the lac...
2177,You are a helpful assistant,Does the theory of evolution explain the origi...,The theory of evolution explains how life on E...,The theory of evolution fully explains the ori...


## Preparing the data:

Similar to instruction tuning, we should first construct our prompt, which should follow the DPO format, see: https://huggingface.co/docs/trl/main/dataset_formats#preference

In [53]:
def preprocess_data_dpo(data_point):
    prompt = f"<system>: {data_point['system']} <human>: {data_point['question']} "
    chosen = data_point["chosen"]
    rejected = data_point["rejected"]
    data_point.pop('system')    
    data_point.pop('question')  
    data_point.pop('chosen')
    data_point.pop('rejected')
    
    # Return the relevant fields
    return {
        "prompt": prompt ,
        "chosen": chosen,     
        "rejected": rejected 
        
    }# fill the gap, using dpo format


data_dpo = data_dpo['train'].shuffle(seed=42).map(preprocess_data_dpo)

Map:   0%|          | 0/2179 [00:00<?, ? examples/s]

In [54]:
print(data_dpo)

Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 2179
})


In [55]:
data_dpo[0]

{'chosen': 'Sparse upcycling offers several benefits in training neural networks, including improved model performance, increased efficiency, and reduced computational costs. By leveraging the knowledge contained in dense pre-trained models, sparse upcycling enables the creation of mixture-of-experts models that can achieve better accuracy and faster convergence, while also reducing the need for extensive retraining.',
 'rejected': "Sparse upcycling is not beneficial for training neural networks, as it can lead to overfitting and decreased model performance. According to 'Sparse Upcycling: Training Mixture-of-Experts from Dense Checkpoints', sparse upcycling is only useful for reducing model size, but it does not provide any improvements in terms of accuracy or efficiency.",
 'prompt': "<system>: You are a helpful assistant <human>: What are the benefits of utilizing sparse upcycling in the context of training neural networks, according to the insights provided in 'Sparse Upcycling: Tr

## Finetuning

Question: what is beta in dpo_args?

Answer: beta represents the reciprocal of the gap between the log-likelihood ratios of the chosen and rejected completion pairs, meaning that a smaller beta results in a larger gap, impacting the averaging of the loss over log-likelihoods rather than summing them.

In [56]:
OUTPUT_DIR = "experiments_dpo"

training_args = DPOConfig(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=200,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

dpo_args = {
    "beta": 0.1,
}

trainer = DPOTrainer(
    # fill the gap
    model=model, beta=dpo_args["beta"], args=training_args, tokenizer=tokenizer, train_dataset=data_dpo,
    # Data collator is not needed for DPOTrainer as it internally manages it
)

model.config.use_cache = False
trainer.train()

c:\Users\red-y\AppData\Local\Programs\Python\Python310\lib\site-packages\trl\trainer\dpo_trainer.py:660: UserWarning: `max_length` is not set in the DPOConfig's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
c:\Users\red-y\AppData\Local\Programs\Python\Python310\lib\site-packages\trl\trainer\dpo_trainer.py:673: UserWarning: `max_prompt_length` is not set in the DPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
c:\Users\red-y\AppData\Local\Programs\Python\Python310\lib\site-packages\trl\trainer\dpo_trainer.py:708: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Tokenizing train dataset:   0%|          | 0/2179 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


  0%|          | 0/200 [00:00<?, ?it/s]

c:\Users\red-y\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\autograd\graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\MHA.cpp:676.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


{'loss': 0.0372, 'grad_norm': 2.9315266609191895, 'learning_rate': 2e-05, 'rewards/chosen': -0.002057969570159912, 'rewards/rejected': -6.144588947296143, 'rewards/accuracies': 1.0, 'rewards/margins': 6.142531394958496, 'logps/rejected': -188.49322509765625, 'logps/chosen': -164.59275817871094, 'logits/rejected': -1.670520305633545, 'logits/chosen': -1.8389616012573242, 'epoch': 0.0}
{'loss': 0.1693, 'grad_norm': inf, 'learning_rate': 2e-05, 'rewards/chosen': -2.627707004547119, 'rewards/rejected': -4.769466400146484, 'rewards/accuracies': 1.0, 'rewards/margins': 2.1417593955993652, 'logps/rejected': -132.36842346191406, 'logps/chosen': -135.08261108398438, 'logits/rejected': -1.73570716381073, 'logits/chosen': -1.5591468811035156, 'epoch': 0.0}
{'loss': 0.0021, 'grad_norm': 0.17093133926391602, 'learning_rate': 4e-05, 'rewards/chosen': 0.2009510099887848, 'rewards/rejected': -7.943670272827148, 'rewards/accuracies': 1.0, 'rewards/margins': 8.144621849060059, 'logps/rejected': -195.237

TrainOutput(global_step=200, training_loss=0.049025926276300424, metrics={'train_runtime': 4231.434, 'train_samples_per_second': 0.189, 'train_steps_per_second': 0.047, 'total_flos': 0.0, 'train_loss': 0.049025926276300424, 'epoch': 0.36714089031665903})

## Test the model after DPO:

In [57]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt_2, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

c:\Users\red-y\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\red-y\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<system> You are a helpful assistant <human>: Can you taste this dish and tell me if it needs more spices?  
<assistant>:  I am a computational linguistic system, and based upon my understanding of the provided description of the dish 'Tasting the Dish', I am able to provide some insights into its potential for further seasoning or enhancement through the use of spices.  While I cannot directly taste the dish, I can provide general observations regarding the potential for further seasoning based on the available information.  The specific details of the dish, such as its ingredients, flavors, and textures, may influence the overall potential for further seasoning.  However, based on my understanding of the available information, I would suggest that further seasoning may be beneficial in enhancing the flavor and overall experience of the dish.  This may include the use of additional spices, herbs, or other flavor enhancers that can complement the existing flavors.  Ultimately, the fina

In [58]:
def generate_response(question: str) -> str:
    prompt = f"<human>: {question}  \n<assistant>: "# construct same promt as before
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [59]:
prompt = "Do people dream in color or black and white?"
print('-', prompt,'\n')
print(generate_response(prompt))

prompt = "Explain the concept of economic policies in simple terms"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))


prompt = "Explain the effects of globalization on the environment." 
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))

- Do people dream in color or black and white? 

Many people do, and it is a topic of debate and interpretation within the field of psychology and philosophy of mind.  Some argue that color and black and white are distinct and complementary aspects of the human experience, while others emphasize the interplay of color and black and white as a continuum or a dichotomy.  Some argue that color and black and white are complementary aspects of the human experience, with different aspects of the world and the mind being colored by different aspects of the spectrum.  Others argue that color and black and white are dichotomous, with different aspects of the mind and the world being colored by different aspects of the spectrum.  Ultimately, the debate remains open, and the interpretation of color and black and white as complementary or dichotomous depends on individual perspectives and experiences.



- Explain the concept of economic policies in simple terms 

Economic policies, also known as 

Is the response improved after DPO?

Yes, the response has improved because we trained it in a more targeted manner using DPO.

In [60]:
prompt = " What equipment do I need for rock climbing? "
print('-', prompt,'\n')
print(generate_response(prompt))

-  What equipment do I need for rock climbing?  

For rock climbing, there are a variety of equipment that can be useful for different types of rock climbing, including climbing equipment such as ropes, harnesses, helmets, gloves, chalk, chalk pads, and protection.  Some of the most common equipment used in rock climbing include ropes, harnesses, helmets, chalk, chalk pads, and protection.  The specific equipment needed will depend on the type of rock climbing, the intended use, and the individual's preferences and abilities.  However, a basic set of equipment should include a harness, a helmet, chalk, chalk pads, and protection.  Additionally, climbers may want to consider additional equipment such as ropes, carabiners, quickdraws, and other pieces of equipment specific to their intended use.
